# Importer les bases de données train et test 

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

## Read csvs
train = gpd.read_file('./train.geojson', index_col=0)
test = gpd.read_file('./test.geojson', index_col=0)
train_y = train['change_type'].apply(lambda x: change_type_map[x])

c:\Users\marce\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(


# Feature analysis and extraction

Work on the geometry feature, create new features that are easier to use in later computations:

perimeter, area, average distance of points to the centroid, variance of the distances of points to the centroid

In [ ]:
def calculate_area(polygon):
    area = polygon.area
    return area

def calculate_perimeter(polygon):
    perimeter = polygon.length
    return perimeter

def calculate_var_distance_to_center(polygon):
    center = polygon.centroid
    exterior_coords = polygon.exterior.coords #on enlève la dernière coordonnée puisque c'est le meme point que le premier
    distances = [center.distance(Point(x, y)) for x, y in exterior_coords]
    return np.var(distances)

def calculate_mean_distance(polygon):
    center = polygon.centroid
    exterior_coords = polygon.exterior.coords
    distances = [center.distance(Point(x, y)) for x, y in exterior_coords]
    return np.mean(distances)

train['area'] = train['geometry'].apply(calculate_area)
train['perimeter'] = train['geometry'].apply(calculate_perimeter)
train['variance_distances_to_center'] = train['geometry'].apply(calculate_var_distance_to_center)
train['calculate_mean_distance'] = train['geometry'].apply(calculate_mean_distance)
train.drop(columns = ['geometry'])

test['area'] = test['geometry'].apply(calculate_area)
test['perimeter'] = test['geometry'].apply(calculate_perimeter)
test['variance_distances_to_center'] = test['geometry'].apply(calculate_var_distance_to_center)
test['calculate_mean_distance'] = test['geometry'].apply(calculate_mean_distance)
test.drop(columns = ['geometry'])


,urban_type,geography_type,img_red_mean_date1,img_green_mean_date1,img_blue_mean_date1,img_red_std_date1,img_green_std_date1,img_blue_std_date1,img_red_mean_date2,img_green_mean_date2,...,change_status_date2,date3,change_status_date3,date4,change_status_date4,index,area,perimeter,variance_distances_to_center,calculate_mean_distance
0,"N,A",Barren Land,203.333944,169.801486,152.200651,15.566236,16.008455,17.248278,187.051282,158.300672,...,Materials Introduced,18-07-2019,Materials Introduced,20-02-2018,Materials Introduced,0,1.017644e-07,0.001293,4.182693e-11,0.000232
1,Rural,Desert,172.061525,147.685474,133.694478,41.475540,44.100325,45.832053,155.780912,135.136855,...,Construction Done,18-07-2019,Construction Done,20-02-2018,Construction Done,1,3.359779e-08,0.000735,1.575673e-11,0.000131
2,Rural,Desert,159.368826,142.102024,127.979757,25.094292,22.485283,23.314198,160.848178,145.442915,...,Construction Midway,18-07-2019,Construction Done,20-02-2018,Construction Done,2,2.582968e-08,0.000649,2.345599e-12,0.000116
3,"N,A",Desert,200.918653,176.145032,160.827045,17.500086,19.342180,20.587617,175.678701,169.652776,...,Construction Done,18-07-2019,Construction Done,20-02-2018,Construction Done,3,4.355159e-07,0.002690,1.664449e-09,0.000488
4,Dense Urban,"Barren Land,Sparse Forest",166.621344,154.779757,149.726308,68.843646,72.361309,71.799610,149.589170,143.869542,...,Construction Midway,18-07-2019,Land Cleared,20-02-2018,Construction Midway,4,9.817937e-08,0.001898,7.654623e-09,0.000290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120521,"N,A","Sparse Forest,Grass Land,Lakes",74.594406,72.981199,62.822552,24.472312,19.897044,19.887582,89.628466,85.693444,...,Land Cleared,19-12-2013,Greenland,29-07-2019,Construction Done,120521,3.984273e-07,0.003063,1.065996e-09,0.000621
120522,"N,A","Sparse Forest,Grass Land,Lakes",58.439701,62.549749,55.780079,21.440513,17.952821,17.289851,78.664141,73.863068,...,Land Cleared,19-12-2013,Greenland,29-07-2019,Construction Done,120522,3.472850e-07,0.002739,4.592515e-10,0.000539
120523,"N,A","Sparse Forest,Grass Land,Lakes",42.641541,50.166332,49.730848,10.641932,9.309863,6.805194,52.940873,60.286320,...,Land Cleared,19-12-2013,Greenland,29-07-2019,Construction Done,120523,3.052487e-07,0.002492,1.198213e-10,0.000482
120524,Sparse Urban,"Coastal,Dense Forest,Grass Land",47.581117,55.697246,49.512326,10.637095,9.854314,7.726375,64.082919,68.935690,...,Land Cleared,19-12-2013,Greenland,29-07-2019,Construction Done,120524,2.733826e-07,0.002335,2.131814e-10,0.000447


Perform One-Hot encoding on the features 'urban_types' and 'geography_types'

In [4]:
urban_types = ['Dense Urban', 'Sparse Urban', 'N,A', 'Urban Slum', 'Rural', 'Industrial']
for urban_type in urban_types:
    #si l'urban type est présent dans la colonne urban_types on ajoute 1 à la colonne urban_type (qui a été ajoutée au df)
    train[urban_type]=train['urban_type'].apply(lambda x: urban_type in x).astype(int) 
    test[urban_type]=test['urban_type'].apply(lambda x: urban_type in x).astype(int) 

In [5]:
geography_types = ['Hills', 'Farms', 'Barren Land', 'Dense Forest', 'N,A', 'Coastal', 'Lakes', 'Desert', 'Sparse Forest', 'River', 'Grass Land', 'Snow']
for geography_type in geography_types:
    train[geography_type]=train['geography_type'].apply(lambda x: geography_type in x).astype(int) 
    test[geography_type]=test['geography_type'].apply(lambda x: geography_type in x).astype(int) 

Work on the date format and sort them chronologically

In [9]:
for i in range(5):
    train[f'date{i}'] = pd.to_datetime(train[f'date{i}'], dayfirst=True, errors='coerce')
    test[f'date{i}'] = pd.to_datetime(test[f'date{i}'], dayfirst=True, errors='coerce')

    
train_sort_by_date=train[['date0','date1','date2','date3','date4']]
test_sort_by_date=test[['date0','date1','date2','date3','date4']]

This code takes the five timestamped observations (date0–date4) and their associated features in the training dataset, reorders the corresponding change_status values and image channel means (green and red) accordingly

In [ ]:
for idx in range(5):
    train[f'date{idx}'] = pd.to_datetime(train[f'date{idx}'])

dates_df = train[[f'date{i}' for i in range(5)]]
dates_df['dates_list'] = dates_df.apply(lambda x: [x[f'date{i}'] for i in range(5)], axis=1)
dates_df['sorted_indices'] = dates_df['dates_list'].apply(lambda x: np.argsort(x))
dates_df['dates_list'] = dates_df['dates_list'].apply(lambda x: [x[i] for i in np.argsort(x)])

change_status_df = train[[f'change_status_date{i}' for i in range(5)]]
dates_df['sorted_change_status'] = change_status_df.apply(lambda x: np.array([x[f'change_status_date{i}'] for i in range(5)]), axis=1)
dates_df['sorted_change_status'] = dates_df.apply(lambda x: x['sorted_change_status'][x['sorted_indices']], axis=1)

# Sort image mean values for green and red channels
green_mean_df = train[[f'img_green_mean_date{i}' for i in range(1, 6)]]
red_mean_df = train[[f'img_red_mean_date{i}' for i in range(1, 6)]]
dates_df['sorted_green_mean'] = green_mean_df.apply(lambda x: np.array([x[f'img_green_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_df['sorted_red_mean'] = red_mean_df.apply(lambda x: np.array([x[f'img_red_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_df['sorted_green_mean'] = dates_df.apply(lambda x: x['sorted_green_mean'][x['sorted_indices']], axis=1)
dates_df['sorted_red_mean'] = dates_df.apply(lambda x: x['sorted_red_mean'][x['sorted_indices']], axis=1)

# Update the original DataFrame with sorted data
for i in range(5):
    train[f'date{i}'] = dates_df['dates_list'].apply(lambda x: x[i])
    train[f'change_status_date{i}'] = dates_df['sorted_change_status'].apply(lambda x: x[i])
    train[f'img_green_mean_date{i+1}'] = dates_df['sorted_green_mean'].apply(lambda x: x[i])
    train[f'img_red_mean_date{i+1}'] = dates_df['sorted_red_mean'].apply(lambda x: x[i])


Idem for the testing set

In [ ]:
for idx in range(5):
    test[f'date{idx}'] = pd.to_datetime(test[f'date{idx}'])

dates_test_df = test[[f'date{i}' for i in range(5)]]
dates_test_df['dates_list'] = dates_test_df.apply(lambda x: [x[f'date{i}'] for i in range(5)], axis=1)
dates_test_df['sorted_indices'] = dates_test_df['dates_list'].apply(lambda x: np.argsort(x))
dates_test_df['dates_list'] = dates_test_df['dates_list'].apply(lambda x: [x[i] for i in np.argsort(x)])

change_status_test_df = test[[f'change_status_date{i}' for i in range(5)]]
dates_test_df['sorted_change_status'] = change_status_test_df.apply(lambda x: np.array([x[f'change_status_date{i}'] for i in range(5)]), axis=1)
dates_test_df['sorted_change_status'] = dates_test_df.apply(lambda x: x['sorted_change_status'][x['sorted_indices']], axis=1)

# Sort image mean values for green and red channels
green_mean_test_df = test[[f'img_green_mean_date{i}' for i in range(1, 6)]]
red_mean_test_df = test[[f'img_red_mean_date{i}' for i in range(1, 6)]]
dates_test_df['sorted_green_mean'] = green_mean_test_df.apply(lambda x: np.array([x[f'img_green_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_test_df['sorted_red_mean'] = red_mean_test_df.apply(lambda x: np.array([x[f'img_red_mean_date{i}'] for i in range(1, 6)]), axis=1)
dates_test_df['sorted_green_mean'] = dates_test_df.apply(lambda x: x['sorted_green_mean'][x['sorted_indices']], axis=1)
dates_test_df['sorted_red_mean'] = dates_test_df.apply(lambda x: x['sorted_red_mean'][x['sorted_indices']], axis=1)

# Update the original DataFrame with sorted data
for i in range(5):
    test[f'date{i}'] = dates_test_df['dates_list'].apply(lambda x: x[i])
    test[f'change_status_date{i}'] = dates_test_df['sorted_change_status'].apply(lambda x: x[i])
    test[f'img_green_mean_date{i+1}'] = dates_test_df['sorted_green_mean'].apply(lambda x: x[i])
    test[f'img_red_mean_date{i+1}'] = dates_test_df['sorted_red_mean'].apply(lambda x: x[i])




We finally arrange the features and creates a function that detects changes over time

In [12]:
def create_dummy_columns(df, date_index):
    dummy_df = pd.get_dummies(df[f'change_status_date{date_index}'])
    columns = [f'{column}_date{date_index}' for column in dummy_df.columns]
    dummy_df.columns = columns
    return dummy_df

dummy_columns = [create_dummy_columns(train, i) for i in range(5)]

train_change_status = pd.concat(dummy_columns, axis=1)

train_df = pd.concat([train, train_change_status], axis=1)

dummy_columns_test = [create_dummy_columns(test, i) for i in range(5)]

test_change_status = pd.concat(dummy_columns_test, axis=1)

test_df = pd.concat([test, test_change_status], axis=1)


Creation of new features that calculate the time intervals between photos taken in the same environment

In [13]:
train_df['passed_time_0']=(train_df['date1']-train_df['date0']).apply(lambda x: x.total_seconds())
train_df['passed_time_1']=(train_df['date2']-train_df['date1']).apply(lambda x: x.total_seconds())
train_df['passed_time_2']=(train_df['date3']-train_df['date2']).apply(lambda x: x.total_seconds())
train_df['passed_time_3']=(train_df['date4']-train_df['date3']).apply(lambda x: x.total_seconds())

test_df['passed_time_0']=(test_df['date1']-test_df['date0']).apply(lambda x: x.total_seconds())
test_df['passed_time_1']=(test_df['date2']-test_df['date1']).apply(lambda x: x.total_seconds())
test_df['passed_time_2']=(test_df['date3']-test_df['date2']).apply(lambda x: x.total_seconds())
test_df['passed_time_3']=(test_df['date4']-test_df['date3']).apply(lambda x: x.total_seconds())

Final processed data saved to perform ML

In [14]:
pd.concat([train_df,train_y],axis=1).to_csv('train_processed.csv')
pd.concat([test_df],axis=1).to_csv('test_processed.csv')

On charge ces fichiers, on met les NaN à 0 par défaut et on fait un XGBoost avec les paramètres qui fonctionnent le mieux pour nous, on enregistre le CSV de submission

In [ ]:
# Charger les données
data = pd.read_csv('train_processed.csv', index_col=0)
data = data.fillna(0)
X = data.iloc[:, :-1].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'change_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])
Y = data.iloc[:, -1]

data_test = pd.read_csv('test_processed.csv', index_col=0)
data_test = data_test.fillna(0)
X_test_p = data_test.iloc[:, :].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])

# Définir les paramètres du modèle XGBoost
xgb_params = {
    'max_depth': 150,        # Profondeur maximale de l'arbre
    'learning_rate': 0.1,   # Taux d'apprentissage
    'n_estimators': 1000,    # Nombre d'arbres à entraîner
    'objective': 'multi:softmax',  # Fonction d'objectif pour une classification multiclasse
    'num_class': 6,         # Nombre de classes dans les données
    'eval_metric': 'mlogloss'     # Métrique d'évaluation
}

# Initialiser et entraîner le modèle XGBoost
xgb = XGBClassifier(**xgb_params)
xgb.fit(X,Y)

y_test = xgb.predict(X_test_p)
pred_df = pd.DataFrame(y_test, columns=['change_type'])
pred_df.to_csv("submission_LMTPE.csv", index=True, index_label='Id')

# Model training and Kaggle submission

Prepare the data for model evaluation

In [30]:
# Charger les données
data = pd.read_csv('train_processed.csv', index_col=0)
data = data.fillna(0)

# Features (X) et target (Y)
X = data.iloc[:, :-1].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'change_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])
Y = data.iloc[:, -1]

# Split train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

Define the XGBoost parameters and perform cross validation to evaluate the model performance

In [31]:
# Définir les paramètres du modèle
xgb_params = {
    'max_depth': 100,
    'learning_rate': 0.01,
    'n_estimators': 1000,
    'objective': 'multi:softmax',
    'num_class': 6,
    'eval_metric': 'mlogloss'
}

# Initialiser et entraîner
xgb = XGBClassifier(**xgb_params)
xgb.fit(X_train, y_train)

# Évaluation
y_pred = xgb.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification report:\n", classification_report(y_val, y_pred))

Accuracy: 0.7762113793685632

Classification report:
               precision    recall  f1-score   support

           0       0.77      0.92      0.84      6302
           1       0.83      0.69      0.75      2861
           2       0.81      0.81      0.81     29687
           3       0.72      0.70      0.71     20085
           4       0.26      0.02      0.04       265
           5       0.00      0.00      0.00        30

    accuracy                           0.78     59230
   macro avg       0.57      0.52      0.53     59230
weighted avg       0.77      0.78      0.77     59230



c:\Users\marce\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\marce\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\marce\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

Final training on the global data and submission on Kaggle

In [29]:
# Charger les données
data = pd.read_csv('train_processed.csv', index_col=0)
data = data.fillna(0)
X = data.iloc[:, :-1].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'change_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])
Y = data.iloc[:, -1]

data_test = pd.read_csv('test_processed.csv', index_col=0)
data_test = data_test.fillna(0)
X_test_p = data_test.iloc[:, :].drop(columns=['N,A', 'Snow', 'urban_type', 'geography_type', 'geometry', 'date0', 'date1', 'date2', 'date3', 'date4','change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3' ,'change_status_date4'])


# Réentraîner sur tout le jeu d’entraînement
xgb_final = XGBClassifier(**xgb_params)
xgb_final.fit(X, Y)

# Prédire sur le test
y_test_pred = xgb_final.predict(X_test_p)

# Sauvegarder la soumission
pred_df = pd.DataFrame(y_test_pred, columns=['change_type'])
pred_df.index.name = 'Id'
pred_df.to_csv("submission_LMTPE.csv")